In [2]:
import os
# os.chdir("../")
%pwd

'/Users/niralpatel/Desktop/Projects/speaking_silence'

In [6]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/niral0901/speaking_silence.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "niral0901"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "147e2b524c36b2df6a579a7e7a16b6b93ac5189b"

In [7]:
import tensorflow as tf

/Users/niralpatel/Desktop/Projects/speaking_silence/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [8]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [22]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    all_params: dict
    mlflow_uri: str


In [29]:
from Speaking_Silence.constants import *
from Speaking_Silence.utils.common import read_yaml, create_directories, save_json
from sklearn.model_selection import train_test_split
from Speaking_Silence.components.model_training import Training
import numpy as np

In [24]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=self.config.training.trained_model_path,
            mlflow_uri=os.environ["MLFLOW_TRACKING_URI"],
            all_params=self.params,
        ) 
        return eval_config

In [25]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import keras
keras_version = keras.__version__

In [38]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.features_test = np.load('artifacts/training/features_test.npy')
        self.labels_test = np.load('artifacts/training/labels_test.npy')

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def validate(self):
        model_evaluation_history = self.model.evaluate(self.features_test, self.labels_test)
        model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history
        return model_evaluation_loss, model_evaluation_accuracy

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.score = self.validate()
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        keras_version = keras.__version__
        print(keras_version)
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model",  registered_model_name="speaking_silence_model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [39]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-04-22 06:35:12: INFO: common] yaml file: config/config.yaml loaded successfully


INFO:Speaking_Silence:yaml file: config/config.yaml loaded successfully


[2024-04-22 06:35:12: INFO: common] yaml file: params.yaml loaded successfully


INFO:Speaking_Silence:yaml file: params.yaml loaded successfully


[2024-04-22 06:35:12: INFO: common] created directory at: artifacts


...

INFO:Speaking_Silence:json file saved at: scores.json


2.15.0


2024/04/22 06:35:15 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/cn/59_k8cws6g95xx26rdq842z80000gn/T/tmpa11ab24p/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/cn/59_k8cws6g95xx26rdq842z80000gn/T/tmpa11ab24p/model/data/model/assets
Successfully registered model 'speaking_silence_model'.
2024/04/22 06:35:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: speaking_silence_model, version 1
Created version '1' of model 'speaking_silence_model'.


In [1]:
from keras.utils import to_categorical

# Assuming yTrain is your original labels
yTrain = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

# One-hot encode the labels
one_hot_yTrain = to_categorical(yTrain, num_classes=15)

# Create a mapping from integer labels to one-hot encoded vectors
label_to_one_hot = {label: one_hot for label, one_hot in zip(yTrain, one_hot_yTrain)}

# Example usage
print(label_to_one_hot[3])

/Users/niralpatel/Desktop/Projects/speaking_silence/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
